![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time, re
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Удобный инструмент для анализа модальности отзыва
from textblob import TextBlob

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
dirnames = set()
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        dirnames.add(dirname)
        
# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 17

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

In [ ]:
print(dirnames)

In [ ]:
cities = pd.read_csv('/kaggle/input/cities-info-for-ta-restaurant-rating/Cities.csv') 


In [ ]:
cities.head()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

Добавляю внешние данные из набора - отношение к стране и численность города

In [ ]:
cities = pd.read_csv('/kaggle/input/cities-info-for-ta-restaurant-rating/Cities.csv') 
joined = data.merge(cities, on='City', how='left')
joined.info()

In [ ]:
# сбрасываю ненужные столбцы
data = joined.drop(['URL_TA','ID_TA'],axis=1)
data.info()

In [ ]:
# Удаление пробелов из названий столбцов
def clean_spaces(df):
    bad_names = []
    substs = {}
    for col in list(df.columns):
        if ' ' in col:
            bad_names.append(col)
    for col in bad_names:
        substs[col] = '_'.join(col.split())
    # substs
    df.rename(columns=substs, inplace=True)
    return df
    

In [ ]:
# Вспомогательная функция для разбора текстового списка видов кухонь
def str2list(s):
    return [ l.strip().strip("'") for l in s.strip('][').split(',') ]

In [ ]:
# Вспомогательные функции для первого разбора столбца отзывов - 
# превращение его в список для дальнейшей обработки.
def strip(text):
    return text[1:-1]
#
def firstsplit(text):
    text = strip(text)
    # print(text)
    if '],' in text:
        pos = text.rfind('],')
        return strip(text[:pos+1]), strip(text[pos+3:])
    else:
        return '', ''
#
def nextsplit(text):
    if '", ' in text:
        pos = text.find('", ')
    elif "', " in text:
        pos = text.find("', ")
    else:
        return strip(text), ''
    return strip(text[:pos+1]), strip(text[pos+3:])
#
def split_review(text):
    # print(text)
    t,d = firstsplit(text)
    t1, t2 = nextsplit(t); d1, d2 = nextsplit(d)
    nr = 2 if len(t1) > 0 and len(t2) > 0 else ( 1 if len(t1) > 0 else 0 )
    return [nr, t1, t2, d1, d2]

In [ ]:
# Анализ текста с целью выясвления отношения и субьективности написавшего.
# Использую усредненные значения по двум отзывам, либо единственное значение по одному, либо все по нулям.
# Еще считается характеристика разброса значений отношения и субьективности - если отзывов два.
def analyse_texts(RevList):
    # RevList[0] - количество опубликованных отзывов
    # RevList[1] - текст отзыва 1
    # RevList[2] - текст отзыва 2
    if RevList[0] == 2:
        tb1 = TextBlob(RevList[1])
        # TextBlob - simples method for sentiment analysis
        p1 = tb1.sentiment.polarity
        s1 = tb1.sentiment.subjectivity
        # The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity).
        # The polarity score is a float within the range [-1.0, 1.0].
        # The subjectivity is a float within the range [0.0, 1.0]
        # where 0.0 is very objective and 1.0 is very subjective.
        # print(RevList[1],p1,s1)
        tb2 = TextBlob(RevList[2])
        p2 = tb2.sentiment.polarity
        s2 = tb2.sentiment.subjectivity
        # print(RevList[2],p2,s2)
        p = (p1+p2)/2      #средняя полярность отзывов
        dp = abs(p2-p1)/2  #разброс полярностей отзывов
        s = (s1+s2)/2      #средняя субьективность отзывов
        ds = abs(s2-s1)/2  #разброс субьективностей отзывов
        return [p, dp, s, ds]
    elif RevList[0] == 1:
        tb1 = TextBlob(RevList[1])
        return [tb1.sentiment.polarity, 0.0, tb1.sentiment.subjectivity, 0.0]
    else:
        return[0.0, 0.0, 0.0, 0.0]

In [ ]:
# Анализ половинки столбца отзывов в виде дат
# Возвращаем самую свежую дату и разброс дат
def analyse_review_dates(RevList):
    # RevList[0] - количество опубликованных отзывов
    # RevList[3] - дата отзыва 1
    # RevList[4] - дата отзыва 2
    if RevList[3] == '':
        c1 = 0
    else:
        c1 = datetime.strptime(RevList[3],"%m/%d/%Y").date().toordinal()
    if RevList[4] == '':
        c2 = 0
    else:
        c2 = datetime.strptime(RevList[4],"%m/%d/%Y").date().toordinal()
    return [ (c2 if c2>c1 else c1), (abs(c2-c1) if c2>0 else 0) ]

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df = clean_spaces(df_input.copy())
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df.drop(['Restaurant_id'], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    # df['Number of Reviews'].fillna(0, inplace=True)
    # тут ваш код по обработке NAN
    # ....
    csfv = df['Cuisine_Style'].value_counts().idxmax()
    df['Cuisine_Style'].fillna(csfv, inplace=True)
    prfv = df['Price_Range'].value_counts().idxmax()
    df['Price_Range'].fillna(prfv, inplace=True)
    nrfv = df['Number_of_Reviews'].median()
    df['Number_of_Reviews'].fillna(nrfv,inplace=True)
    df['Reviews'].fillna('[[], []]',inplace=True)

    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    # df = pd.get_dummies(df, columns=[ 'City',], dummy_na=True)
    # тут ваш код не Encoding фитчей
    # ....
    city_columns = pd.get_dummies(df['City'])
    df = df.join(city_columns).drop('City',axis=1)
    country_columns = pd.get_dummies(df['Country_Code'])
    df = df.join(country_columns).drop('Country_Code',axis=1)
    df['Log_Population']=np.log10(df.Population)
    df = df.drop(['Population'],axis=1)
    
    # Словарик и преобразование диапазона цен в набор значений
    # Проверка показала что это более эффективное решение, чем one_hot кодирование. Быстрее и ошибки меньше.
    PriceRanges = { '$':1.0, '$$$$': 3.0, '$$ - $$$': 2.0 }
    df['PRange']= df['Price_Range'].apply(lambda c: PriceRanges[c])
    df = df.drop(['Price_Range'],axis=1)

    # Превращаю поле Cuisine_Style в набор индексов и одну количественную переменную
    # Вспомогательный столбец со списком кухонь
    df['CStyles'] = df['Cuisine_Style'].apply(str2list)
    # СStyles - вспомогательный датафрейм для использования explode
    # В одной ячейке может содержаться несколько названий кухонь,  поэтому сначала делаю их список через explode
    # а потом добавляю столбцы с названиями и превращаю их в индексы
    CStyles = df['Cuisine_Style'].unique()
    CStyles = pd.Series(CStyles)
    CStyles = CStyles.apply(str2list)
    CStyles = CStyles.explode()
    # CNames - список кухонь
    CNames = CStyles.unique()
    for st in CNames:
        df[st] = df['CStyles'].apply(lambda x: 1 if st in x else 0)
    df['Num_Cuisines'] = df['CStyles'].apply(len) # это уже дамми переменная с количеством разных кухонь
    df = df.drop(['CStyles', 'Cuisine_Style'], axis=1)
    df = clean_spaces(df) # в названиях кухонь встречаются пробелы
    
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # ....
    # Разбор колонки отзывов в два столбца со списком значений отзывов и
    # с количеством отзывов для будущего использоваиния
    df.head(5)
    df['RL'] = df['Reviews'].apply(split_review)
    df['Num_Disp_Rev'] = df['RL'].apply(lambda x: x[0])
    # Разбор текстовой половины отзывов
    df['TAL'] = df['RL'].apply(analyse_texts)
    df['Review_polarity'] = df['TAL'].apply(lambda x: x[0])
    df['Review_polarity_variation'] = df['TAL'].apply(lambda x: x[1])
    df['Review_subjectivity'] = df['TAL'].apply(lambda x: x[2])
    df['Review_subjectivity_variation'] = df['TAL'].apply(lambda x: x[3])
    df = df.drop(['TAL'], axis=1)
    # Разбор даты отзывов
    df['RDAL'] = df['RL'].apply(analyse_review_dates)
    df['Recent_Review_date'] = df['RDAL'].apply(lambda x: x[0])
    df['Review_dates_span'] = df['RDAL'].apply(lambda x: x[1])
    df = df.drop(['RDAL'],axis=1)
    #Удивительным образом заполнение пропусков даты обзора медианой а не нулем дает чуть лучшее значение.
    # А вот нормализация значений заметным образом не сказывается на итоге.
    Med_Rec_Rev_Date = df.loc[df.Recent_Review_date>0,'Recent_Review_date'].median()
    df['Recent_Review_date'] = df['Recent_Review_date'].apply(lambda x: x if x > 0 else Med_Rec_Rev_Date)
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df.columns if df[s].dtypes == 'object']
    df.drop(object_columns, axis = 1, inplace=True)
    
    return df

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)
y_pred2 = (y_pred*2).round()/2

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE: ', metrics.mean_absolute_error(y_test, y_pred))
print('MAE2:', metrics.mean_absolute_error(y_test, y_pred2))


In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission = (predict_submission*2).round()/2
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
